In [ ]:
#!pip install transformers


In [ ]:
import pandas as pd
df=pd.read_excel("auto_topic_labels - kamran.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   topic_no      18 non-null     int64 
 1   labels        18 non-null     object
 2   expert label  18 non-null     object
dtypes: int64(1), object(2)
memory usage: 560.0+ bytes


In [ ]:
import nltk
from collections import Counter
from transformers import pipeline
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')
nltk.download('wordnet')

# Customize stop words list
stop_words = set(stopwords.words('english'))
stop_words.update(['approach', 'a', 'an', 'and', 'the','novel','new','future'])

# Lemmatize words
lemmatizer = WordNetLemmatizer()

def label_analysis(labels):
    # Word frequency analysis
    all_words = []
    for label in labels:
        words = word_tokenize(label.lower())
        tagged_words = nltk.pos_tag(words)
        for word, tag in tagged_words:
            if tag.startswith('J'):
                lemmatized_word = lemmatizer.lemmatize(word)
                if lemmatized_word not in stop_words and lemmatized_word.isalpha():
                    all_words.append(lemmatized_word)
            else:
                lemmatized_word = lemmatizer.lemmatize(word)
                if lemmatized_word not in stop_words and lemmatized_word.isalpha():
                    all_words.append(lemmatized_word)

    word_counts = Counter(all_words)
    most_common_words = word_counts.most_common(4)

    # Named entity recognition
    ner_labels = []
    for label in labels:
        words = word_tokenize(label)
        tagged_words = nltk.pos_tag(words)
        chunked_words = nltk.ne_chunk(tagged_words)
        for chunk in chunked_words:
            if hasattr(chunk, 'label') and chunk.label() == 'GPE':
                ner_labels.append(' '.join(c[0] for c in chunk.leaves()))

    ner_counts = Counter(ner_labels)
    most_common_ner = ner_counts.most_common(1)

    # Generate general label
    keywords = [word for word, count in most_common_words if word not in ['in', '-']]
    general_label = ' '.join(keywords)

    # Transfer learning
    nlp = pipeline('ner', grouped_entities=True)
    tokens = [nlp(label) for label in labels]
    entities = set()
    for token in tokens:
        for entity in token:
            if entity['entity_group'] == 'ORG':
                entities.add(entity['word'])

    if len(entities) > 0:
        # Generate general label
        keywords = [word for word, count in most_common_words if word not in ['in', '-']]
        if len(keywords) > 4:
            keywords = keywords[:4]

    return general_label

df = pd.read_excel("auto_topic_labels - kamran.xlsx")

general_labels = []

for i in range(len(df)):
    label_list = [label.strip() for label in df.loc[i, 'labels'].split(';')]
    general_label = label_analysis(label_list)
    general_labels.append(general_label)

df['general_label'] = general_labels

df


No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll0

,topic_no,labels,expert label,general_label
0,1,['Enhanced Image Restoration - A Novel Approac...,underwater imaging,image underwater enhancement restoration
1,2,['Energy Efficiency in Underwater Sensor Netwo...,sensor networks,sensor network wireless efficiency
2,3,"['Robotic Underwater Vehicles', 'The Future of...",autonomous underwater vehicles (AUVs),underwater vehicle development navigation
3,4,['Underwater Panorama Image Mosaic Technology ...,underwater image processing,image underwater compression method
4,5,['Sonar Resolution - A New Approach to Sonar D...,sonar detection,sonar detection acoustical sensor
5,6,['Using Modeling and Simulation to Support the...,sensing systems,marine development underwater system
6,7,"['Maritime Security - A New Approach', 'The Fu...",maritime security,security marine threat underwater
7,8,['Detecting Intrusions Using Machine Learning'...,ML for underwater object detection and classif...,underwater learning object tracking
8,9,['Underwater Acoustic Sensor Networks - A Rese...,network protocol and routing,network sensor acoustic protocol
9,10,"['Detecting Clusters of a Ship', 'A Novel Anom...",maritime anomaly detection,anomaly detection marine modeling


In [ ]:
# Customize stop words list
stop_words = set(stopwords.words('english'))
stop_words.update(['approach', 'a', 'an', 'and', 'the','novel','new','future'])

def label_analysis(labels):
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    all_words = [lemmatizer.lemmatize(word) for word in word_tokenize(' '.join(labels).lower())]

    # Word frequency analysis
    filtered_words = [word for word in all_words if word not in stop_words and word.isalpha()]
    word_counts = Counter(filtered_words)
    most_common_words = word_counts.most_common()

    # Named entity recognition
    ner_labels = []
    for label in labels:
        words = word_tokenize(label)
        tagged_words = nltk.pos_tag(words)
        chunked_words = nltk.ne_chunk(tagged_words)
        for chunk in chunked_words:
            if hasattr(chunk, 'label') and chunk.label() == 'GPE':
                ner_labels.append(' '.join(c[0] for c in chunk.leaves()))

    ner_counts = Counter(ner_labels)
    most_common_ner = ner_counts.most_common(1)

    # Generate general label
    topic_word_count = len(most_common_words)
    if topic_word_count > 2:
        keywords = [word for word, count in most_common_words[:3] if word not in ['in', '-']]
    else:
        keywords = [word for word, count in most_common_words if word not in ['in', '-']]

    general_label = ' '.join(keywords)

    # Transfer learning
    nlp = pipeline('ner', grouped_entities=True)
    tokens = [nlp(label) for label in labels]
    entities = set()
    for token in tokens:
        for entity in token:
            if entity['entity_group'] == 'ORG':
                entities.add(entity['word'])

    if len(entities) > 0:
        # Generate general label
        if topic_word_count > 4:
            keywords = [word for word, count in most_common_words[:4] if word not in ['in', '-']]
        else:
            keywords = [word for word, count in most_common_words if word not in ['in', '-']]

    return general_label

df = pd.read_excel("auto_topic_labels - kamran.xlsx")

general_labels = []

for i in range(len(df)):
    label_list = [label.strip() for label in df.loc[i, 'labels'].split(';')]
    general_label = label_analysis(label_list)
    general_labels.append(general_label)

df['general_label'] = general_labels

df


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without

,topic_no,labels,expert label,general_label
0,1,['Enhanced Image Restoration - A Novel Approac...,underwater imaging,image underwater enhancement
1,2,['Energy Efficiency in Underwater Sensor Netwo...,sensor networks,sensor network wireless
2,3,"['Robotic Underwater Vehicles', 'The Future of...",autonomous underwater vehicles (AUVs),underwater vehicle development
3,4,['Underwater Panorama Image Mosaic Technology ...,underwater image processing,image underwater compression
4,5,['Sonar Resolution - A New Approach to Sonar D...,sonar detection,sonar detection acoustical
5,6,['Using Modeling and Simulation to Support the...,sensing systems,marine development underwater
6,7,"['Maritime Security - A New Approach', 'The Fu...",maritime security,security marine threat
7,8,['Detecting Intrusions Using Machine Learning'...,ML for underwater object detection and classif...,underwater learning object
8,9,['Underwater Acoustic Sensor Networks - A Rese...,network protocol and routing,network sensor acoustic
9,10,"['Detecting Clusters of a Ship', 'A Novel Anom...",maritime anomaly detection,anomaly detection marine
